## MLflow's Model Registry

In [1]:
from mlflow.tracking import MlflowClient


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

### Interacting with the MLflow tracking server

The `MlflowClient` object allows us to interact with...
- an MLflow Tracking Server that creates and manages experiments and runs.
- an MLflow Registry Server that creates and manages registered models and model versions. 

To instantiate it we need to pass a tracking URI and/or a registry URI

In [3]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

client.list_artifacts

<bound method MlflowClient.list_artifacts of <mlflow.tracking.client.MlflowClient object at 0x70be61d73bf0>>

In [4]:
client.create_experiment(name="my-cool-experiment")

'2'

Let's check the latest versions for the experiment with id `1`...

In [5]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [6]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 55b3b7bc470d4acfaeae5d5265a061c1, rmse: 6.3184


### Interacting with the Model Registry

In this section We will use the `MlflowClient` instance to:

1. Register a new version for the experiment `nyc-taxi-regressor`
2. Retrieve the latests versions of the model `nyc-taxi-regressor` and check that a new version `4` was created.
3. Transition the version `4` to "Staging" and adding annotations to it.

In [7]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [9]:


run_id = "55b3b7bc470d4acfaeae5d5265a061c1"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
Created version '1' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1744920250798, current_stage='None', description=None, last_updated_timestamp=1744920250798, name='nyc-taxi-regressor', run_id='55b3b7bc470d4acfaeae5d5265a061c1', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment_tracking/mlruns/1/55b3b7bc470d4acfaeae5d5265a061c1/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [10]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


/tmp/ipykernel_13071/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [11]:
model_version = 1
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_13071/1600074043.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1744920250798, current_stage='Staging', description=None, last_updated_timestamp=1744920266074, name='nyc-taxi-regressor', run_id='55b3b7bc470d4acfaeae5d5265a061c1', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment_tracking/mlruns/1/55b3b7bc470d4acfaeae5d5265a061c1/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [12]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1744920250798, current_stage='Staging', description='The model version 1 was transitioned to Staging on 2025-04-17', last_updated_timestamp=1744920269369, name='nyc-taxi-regressor', run_id='55b3b7bc470d4acfaeae5d5265a061c1', run_link=None, source='/workspaces/mlops-zoomcamp/02-experiment_tracking/mlruns/1/55b3b7bc470d4acfaeae5d5265a061c1/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

### Comparing versions and selecting the new "Production" model

In the last section, we will retrieve models registered in the model registry and compare their performance on an unseen test set. The idea is to simulate the scenario in which a deployment engineer has to interact with the model registry to decide whether to update the model version that is in production or not.

These are the steps:

1. Load the test dataset, which corresponds to the NYC Green Taxi data from the month of March 2021.
2. Download the `DictVectorizer` that was fitted using the training data and saved to MLflow as an artifact, and load it with pickle.
3. Preprocess the test set using the `DictVectorizer` so we can properly feed the regressors.
4. Make predictions on the test set using the model versions that are currently in the "Staging" and "Production" stages, and compare their performance.
5. Based on the results, update the "Production" model version accordingly.


**Note: the model registry doesn't actually deploy the model to production when you transition a model to the "Production" stage, it just assign a label to that model version. You should complement the registry with some CI/CD code that does the actual deployment.**

In [18]:
from sklearn.metrics import mean_squared_error
import pandas as pd



def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [20]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [21]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

/home/codespace/.python/current/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/workspaces/mlops-zoomcamp/02-experiment_tracking/preprocessor'

In [22]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [23]:
X_test = preprocess(df, dv)

In [24]:
target = "duration"
y_test = df[target].values

In [32]:
model_name = "nyc-exp"

In [31]:
%time test_model(name="nyc-exp", stage="Staging", X_test=X_test, y_test=y_test)

TypeError: got an unexpected keyword argument 'squared'

In [33]:
%time test_model(name=model_name, stage="Staging", X_test=X_test, y_test=y_test)

TypeError: got an unexpected keyword argument 'squared'

In [34]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_13071/1316468422.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1744919580015, current_stage='Production', description='', last_updated_timestamp=1744920862906, name='nyc-exp', run_id='55b3b7bc470d4acfaeae5d5265a061c1', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment_tracking/mlruns/1/55b3b7bc470d4acfaeae5d5265a061c1/artifacts/models_mlflow', status='READY', status_message=None, tags={}, user_id=None, version=1>